# ETL pipeline

## Precedures: 
1.  [**Extracting**](#1)

2.  [**Transforming and cleaning**](#2)

3.  [**Loading**](#3)

4. [**Testing**](#4)

<a id="1"></a>
## Extract data for original dataset

In [1]:
import pandas as pd
import re

In [2]:
# load different origin dataset 
# merge together by same column id
messages = pd.read_csv('disaster_messages.csv')
categories = pd.read_csv('disaster_categories.csv')

df = messages.merge(categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [3]:
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


<a id="2"></a>
## Transform categories into new features

In [5]:
# extract label's name for a categories value
columns = []
for i in df.categories[0].split(';'):
    columns.append(re.sub('[^a-zA-Z]', ' ', i).strip())
    
print(columns)

['related', 'request', 'offer', 'aid related', 'medical help', 'medical products', 'search and rescue', 'security', 'military', 'child alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing people', 'refugees', 'death', 'other aid', 'infrastructure related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid centers', 'other infrastructure', 'weather related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other weather', 'direct report']


In [6]:
# expand categories into new features by using str.split()
# rename new column's name by different labels
new_columns = [ re.sub('[^a-zA-Z]', ' ', i).strip() for i in df.categories[0].split(';')]
categories_df = df.categories.str.split(';', expand=True)
categories_df.columns = new_columns
categories_df.head()

,related,request,offer,aid related,medical help,medical products,search and rescue,security,military,child alone,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [7]:
# remove anything except int value and replace with empty space
# reset colume type as str
for column in categories_df:
    categories_df[column] = categories_df[column].apply(lambda x: re.sub('[^0-9]', '', x)).astype('str')
    
categories_df.head()    

,related,request,offer,aid related,medical help,medical products,search and rescue,security,military,child alone,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.shape

(26386, 5)

In [9]:
categories_df.shape

(26386, 36)

In [10]:
# concatenate origin df and new feature df
# remove old categories column
df = pd.concat([df, categories_df], axis=1)
df = df.drop('categories', axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid related,medical help,medical products,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
for col in df.columns[4:]:
    print(col)
    print(df[col].unique())

related
['1' '0' '2']
request
['0' '1']
offer
['0' '1']
aid related
['0' '1']
medical help
['0' '1']
medical products
['0' '1']
search and rescue
['0' '1']
security
['0' '1']
military
['0' '1']
child alone
['0']
water
['0' '1']
food
['0' '1']
shelter
['0' '1']
clothing
['0' '1']
money
['0' '1']
missing people
['0' '1']
refugees
['0' '1']
death
['0' '1']
other aid
['0' '1']
infrastructure related
['0' '1']
transport
['0' '1']
buildings
['0' '1']
electricity
['0' '1']
tools
['0' '1']
hospitals
['0' '1']
shops
['0' '1']
aid centers
['0' '1']
other infrastructure
['0' '1']
weather related
['0' '1']
floods
['0' '1']
storm
['0' '1']
fire
['0' '1']
earthquake
['0' '1']
cold
['0' '1']
other weather
['0' '1']
direct report
['0' '1']


For related feature, it seemed like there is not a binary values, so we need to remove it.

In [12]:
# remove the row which have values as 2 in related feature
df = df[df['related']!='2']

for col in df.columns[4:]:
    print(col)
    print(df[col].unique())

related
['1' '0']
request
['0' '1']
offer
['0' '1']
aid related
['0' '1']
medical help
['0' '1']
medical products
['0' '1']
search and rescue
['0' '1']
security
['0' '1']
military
['0' '1']
child alone
['0']
water
['0' '1']
food
['0' '1']
shelter
['0' '1']
clothing
['0' '1']
money
['0' '1']
missing people
['0' '1']
refugees
['0' '1']
death
['0' '1']
other aid
['0' '1']
infrastructure related
['0' '1']
transport
['0' '1']
buildings
['0' '1']
electricity
['0' '1']
tools
['0' '1']
hospitals
['0' '1']
shops
['0' '1']
aid centers
['0' '1']
other infrastructure
['0' '1']
weather related
['0' '1']
floods
['0' '1']
storm
['0' '1']
fire
['0' '1']
earthquake
['0' '1']
cold
['0' '1']
other weather
['0' '1']
direct report
['0' '1']


In [13]:
# check any duplicated rows
df.duplicated().sum()

154

In [14]:
# remove any duplicated rows
df = df.drop_duplicates()
df.duplicated().sum()

0

<a id="3"></a>
## Load new dataset into database

In [15]:
# load clean df into database, self setting table name
import sqlite3

conn = sqlite3.connect('Disaster_database.db')
df.to_sql('Disaster_database', con=conn, if_exists='replace', index=False)

C:\Users\EllenChen\anaconda3\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [16]:
# read df from database table
db = pd.read_sql('select * from Disaster_database', con=conn)
print(db.shape)
db.head()

(26028, 40)


,id,message,original,genre,related,request,offer,aid related,medical help,medical products,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<a id="4"></a>
## Test pipeline output

Loading dataset which conducted by **process_data.py** pipeline

In [17]:
conn = sqlite3.connect('Disaster_database.db')
pipe_df = pd.read_sql('select * from overall', con=conn)
pipe_df.head()

,id,message,original,genre,related,request,offer,aid related,medical help,medical products,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df.head()

,id,message,original,genre,related,request,offer,aid related,medical help,medical products,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
cat_df = df[df.columns[4:]]
cat_df.head()

,related,request,offer,aid related,medical help,medical products,search and rescue,security,military,child alone,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
for col in cat_df.columns:
    print(col)
    print(cat_df[col].unique())

related
['1' '0']
request
['0' '1']
offer
['0' '1']
aid related
['0' '1']
medical help
['0' '1']
medical products
['0' '1']
search and rescue
['0' '1']
security
['0' '1']
military
['0' '1']
child alone
['0']
water
['0' '1']
food
['0' '1']
shelter
['0' '1']
clothing
['0' '1']
money
['0' '1']
missing people
['0' '1']
refugees
['0' '1']
death
['0' '1']
other aid
['0' '1']
infrastructure related
['0' '1']
transport
['0' '1']
buildings
['0' '1']
electricity
['0' '1']
tools
['0' '1']
hospitals
['0' '1']
shops
['0' '1']
aid centers
['0' '1']
other infrastructure
['0' '1']
weather related
['0' '1']
floods
['0' '1']
storm
['0' '1']
fire
['0' '1']
earthquake
['0' '1']
cold
['0' '1']
other weather
['0' '1']
direct report
['0' '1']
